In [17]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../../src')
from ecomplexity import ecomplexity

# 小数点以下 桁数 6
pd.options.display.float_format = '{:.3f}'.format


In [2]:
import initial_condition
from analysis import weight


In [3]:
global data_dir, filter_dir, output_dir
data_dir = '../../data/interim/internal/filtered_before_agg/'
filter_dir = '../../data/interim/internal/filter_after_agg/'
output_dir = '../../data/interim/internal/filtered_after_agg/'


In [4]:
# 初期条件
ar = initial_condition.AR
year_style = initial_condition.YEAR_STYLE

year_start = initial_condition.YEAR_START
year_end = initial_condition.YEAR_END
year_range = initial_condition.YEAR_RANGE

classification = initial_condition.CLASSIFICATION
class_weight = initial_condition.CLASS_WEIGHT
applicant_weight = initial_condition.APPLICANT_WEIGHT

extract_population = initial_condition.EXTRACT_POPULATION
top_p_or_num = initial_condition.TOP_P_OR_NUM


In [5]:
# 全体
all_df = pd.read_csv(f'{data_dir}addedclassification.csv', 
                     encoding='utf-8', 
                     sep=',', 
                     usecols=['reg_num', 
                              'right_person_name', 
                              f'{ar}_{year_style}', 
                              f'{classification}'], 
                     dtype={'reg_num': str, 
                            'right_person_name': str, 
                            f'{ar}_{year_style}': np.int64, 
                            f'{classification}': str})

all_df = all_df[all_df[f'{ar}_{year_style}'].isin(range(year_start, year_end+1))]\
               .drop_duplicates()\
# display(all_df.head())


# 各期間
sep_year_df_dict = {}

for year in range(year_start, year_end+1, year_range):
    sep_year_df_dict[f'{year}-{year+year_range-1}'] = all_df[all_df[f'{ar}_year'].isin(range(year, year+year_range))]


In [6]:
# 特許分類による重みづけ
# 全体
if class_weight == 'fraction':
    all_df = weight.by_classification(all_df, classification)
elif class_weight == 'duplication':
    all_df['class_weight'] = 1
all_df[f'{ar}_{year_style}_period'] = f'{year_start}-{year_end}'


# 期間ごと
# sep_year_df_dict = {}
# sep_year_reg_num_top_df_dict = {}
for period, sep_year_df in sep_year_df_dict.items():
    if class_weight == 'fraction':
        sep_year_df_dict[period] = weight.by_classification(sep_year_df, classification)
    elif class_weight == 'duplication':
        sep_year_df_dict[period] = sep_year_df.groupby(['right_person_name', classification])[['reg_num']].nunique().reset_index(drop=False)
    sep_year_df_dict[period][f'{ar}_{year_style}_period'] = period

# 共同出願の重みづけ
# 全体
if applicant_weight == 'fraction':
    all_df = weight.by_applicant(all_df)
elif applicant_weight == 'duplication':
    all_df['applicant_weight'] = 1
all_df[f'{ar}_{year_style}_period'] = f'{year_start}-{year_end}'


# 期間ごと
# sep_year_df_dict = {}
# sep_year_reg_num_top_df_dict = {}
for period, sep_year_df in sep_year_df_dict.items():
    if applicant_weight == 'fraction':
        sep_year_df_dict[period] = weight.by_applicant(sep_year_df)
    elif applicant_weight == 'duplication':
        sep_year_df_dict[period]['applicant_weight'] = 1
    sep_year_df_dict[period][f'{ar}_{year_style}_period'] = period



In [7]:
all_reg_num_df = all_df.copy()
all_reg_num_df['reg_num'] = 1 / all_reg_num_df['class_weight'] / all_reg_num_df['applicant_weight']
all_reg_num_df = all_reg_num_df.groupby([f'{ar}_{year_style}_period', 'right_person_name', classification])[['reg_num']]\
                               .sum().reset_index(drop=False)\
                               .sort_values(['reg_num'], ascending=[False])
sep_year_reg_num_df_dict = sep_year_df_dict.copy()
for period, sep_year_reg_num_df in sep_year_reg_num_df_dict.items():
    sep_year_reg_num_df['reg_num'] = 1 / sep_year_reg_num_df['class_weight'] / sep_year_reg_num_df['applicant_weight']
    sep_year_reg_num_df = sep_year_reg_num_df.groupby([f'{ar}_{year_style}_period', 'right_person_name', classification])[['reg_num']]\
                                             .sum().reset_index(drop=False)\
                                             .sort_values(['reg_num'], ascending=[False])
    sep_year_reg_num_df_dict[period] = sep_year_reg_num_df
sep_year_reg_num_df = pd.concat([sep_year_reg_num_df for sep_year_reg_num_df in sep_year_reg_num_df_dict.values()], axis='index', ignore_index=True)
sep_year_reg_num_df


,app_year_period,right_person_name,schmoch_35,reg_num
0,1981-1990,パナソニツクホールデイングス株式会社,1,9089.785714
1,1981-1990,パナソニツクホールデイングス株式会社,2,8859.985714
2,1981-1990,キヤノン株式会社,9,7249.333333
3,1981-1990,富士通株式会社,6,6323.016667
4,1981-1990,パナソニツクホールデイングス株式会社,30,5119.416667
...,...,...,...,...
248530,2001-2010,北陸電力株式会社,9,0.071429
248531,2001-2010,日本電計株式会社,31,0.071429
248532,2001-2010,五洋建設株式会社,31,0.071429
248533,2001-2010,ＭＨＩエンジニアリング株式会社,19,0.066667


In [8]:
# フィルタリング
reg_num_filter_df = pd.read_csv(f'{filter_dir}{ar}_{year_style}_{extract_population}_reg_num_top_{top_p_or_num[0]}_{top_p_or_num[1]}.csv',
                                encoding='utf-8',
                                sep=',', 
                                usecols=[f'{ar}_{year_style}_period', 'right_person_name'],
                                dtype=str)
# reg_num_filter_df

In [18]:
if extract_population == 'all':
    all_reg_num_top_df = pd.merge(
        all_reg_num_df,
        reg_num_filter_df,
        on=[f'{ar}_{year_style}_period', 'right_person_name'],
        how='inner',
    )
    sep_year_reg_num_top_df = pd.merge(
        sep_year_reg_num_df,
        reg_num_filter_df[['right_person_name']],
        on=['right_person_name'], 
        how='inner'
    )
sep_year_reg_num_top_df

reg_num_top_df = pd.concat([all_reg_num_top_df, sep_year_reg_num_top_df], axis='index', ignore_index=True)
reg_num_top_df


,app_year_period,right_person_name,schmoch_35,reg_num
0,1981-2010,キヤノン株式会社,9,23715.000
1,1981-2010,キヤノン株式会社,28,9984.333
2,1981-2010,キヤノン株式会社,2,9520.833
3,1981-2010,キヤノン株式会社,6,7437.333
4,1981-2010,キヤノン株式会社,3,6348.500
...,...,...,...,...
11673,2001-2010,国立研究開発法人産業技術総合研究所,4,19.833
11674,2001-2010,国立研究開発法人産業技術総合研究所,18,15.583
11675,2001-2010,国立研究開発法人産業技術総合研究所,34,14.667
11676,2001-2010,国立研究開発法人産業技術総合研究所,7,11.000


In [19]:
reg_num_top_df.to_csv(f'{output_dir}{ar}_{year_style}_{extract_population}_{top_p_or_num[0]}_{top_p_or_num[1]}.csv', 
                      encoding='utf-8', 
                      sep=',', 
                      index=False)
